Implement a CNN on Tomato dataset using batch sizes of 32 and 64 separately. Keep the learning 
rate fixed at 0.0001 and compare results. 

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os

IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZES = [32, 64]
EPOCHS = 20
LEARNING_RATE = 0.0001

train_dir = '/home/adithya/Deep_Learning/DL3_Dataset/Tomato/Train'
val_dir = '/home/adithya/Deep_Learning/DL3_Dataset/Tomato/Val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def plot_history(histories, batch_sizes):
    plt.figure(figsize=(12, 5))
    for i, history in enumerate(histories):
        plt.subplot(1, 2, 1)
        plt.plot(history.history['val_accuracy'], label=f'Batch {batch_sizes[i]}')
        plt.title('Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['val_loss'], label=f'Batch {batch_sizes[i]}')
        plt.title('Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    if not os.path.exists(train_dir):
        raise FileNotFoundError(f"Training directory '{train_dir}' does not exist. Please check your dataset path.")

    class_folders = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
    if not class_folders:
        raise ValueError(f"No class subfolders found in '{train_dir}'. Please organize your dataset as '{train_dir}/class_name/'.")
    num_classes = len(class_folders)
    print(f'Number of classes: {num_classes}')

    histories = []

    for batch_size in BATCH_SIZES:
        print(f"\nTraining with batch size: {batch_size}")
        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(IMG_HEIGHT, IMG_WIDTH),
            batch_size=batch_size,
            class_mode='categorical'
        )
        val_generator = val_datagen.flow_from_directory(
            val_dir,
            target_size=(IMG_HEIGHT, IMG_WIDTH),
            batch_size=batch_size,
            class_mode='categorical'
        )

        model = build_cnn_model((IMG_HEIGHT, IMG_WIDTH, 3), num_classes)
        model.compile(
            optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        history = model.fit(
            train_generator,
            epochs=EPOCHS,
            validation_data=val_generator,
            verbose=1
        )
        histories.append(history)

    plot_history(histories, BATCH_SIZES)

    for i, batch_size in enumerate(BATCH_SIZES):
        val_acc = histories[i].history['val_accuracy'][-1]
        print(f"Batch Size {batch_size}: Final Validation Accuracy = {val_acc:.4f}")

Number of classes: 6

Training with batch size: 32
Found 11108 images belonging to 6 classes.
Found 2495 images belonging to 6 classes.
Epoch 1/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 75s 200ms/step - accuracy: 0.3344 - loss: 1.5898 - val_accuracy: 0.6289 - val_loss: 0.9397
Epoch 2/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 72s 207ms/step - accuracy: 0.6631 - loss: 0.9391 - val_accuracy: 0.6882 - val_loss: 0.8038
Epoch 3/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 77s 222ms/step - accuracy: 0.7343 - loss: 0.7498 - val_accuracy: 0.6986 - val_loss: 0.7754
Epoch 4/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 72s 207ms/step - accuracy: 0.7676 - loss: 0.6504 - val_accuracy: 0.7535 - val_loss: 0.7169
Epoch 5/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 75s 214ms/step - accuracy: 0.7901 - loss: 0.6019 - val_accuracy: 0.7483 - val_loss: 0.7239
Epoch 6/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 66s 191ms/step - accuracy: 0.8071 - loss: 0.5307 - val_accuracy: 0.7218 - val_loss: 0.8559
Epoch 7/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 52s 149ms/step - accuracy: 0.8219 - lo

2025-05-22 20:53:37.429847: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,128,128]{3,2,1,0} %bitcast.4699, f32[32,3,3,3]{3,2,1,0} %bitcast.4706, f32[32]{0} %bitcast.5244), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_2_1/conv2d_6_1/convolution" source_file="/home/adithya/Deep_Learning/tensorflow_env/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-22 20:53:37.530459: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549

 29/174 ━━━━━━━━━━━━━━━━━━━━ 32s 224ms/step - accuracy: 0.1698 - loss: 1.7956

2025-05-22 20:53:48.732776: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.9 = (f32[36,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[36,3,128,128]{3,2,1,0} %bitcast.4699, f32[32,3,3,3]{3,2,1,0} %bitcast.4706, f32[32]{0} %bitcast.5244), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_2_1/conv2d_6_1/convolution" source_file="/home/adithya/Deep_Learning/tensorflow_env/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-22 20:53:48.816814: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549

174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.2512 - loss: 1.7078

2025-05-22 20:54:24.626880: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,128,128]{3,2,1,0} %bitcast.531, f32[32,3,3,3]{3,2,1,0} %bitcast.538, f32[32]{0} %bitcast.540), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_2_1/conv2d_6_1/convolution" source_file="/home/adithya/Deep_Learning/tensorflow_env/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-22 20:54:24.704037: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] O

174/174 ━━━━━━━━━━━━━━━━━━━━ 53s 270ms/step - accuracy: 0.2517 - loss: 1.7070 - val_accuracy: 0.4689 - val_loss: 1.2538
Epoch 2/20
 95/174 ━━━━━━━━━━━━━━━━━━━━ 19s 252ms/step - accuracy: 0.5335 - loss: 1.2650

KeyboardInterrupt: 